In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
import mmap
import random
import pickle
import argparse
import lzma
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
#parameters
block_size = 8
batch_size = 4
max_iters = 1000
# eval_iters = 2500
learning_rate = 3e-4
eval_iters = 250

In [ ]:
with open(r'C:\Users\tiakh\AppData\Roaming\jupyter\kernels\cuda\Wiz_of_oz.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
# print(test[:200])
chars = sorted(set(text))
vocab_size = len(chars)
# print(vocab_size)

In [ ]:
#Creating a HashMap that maps to character to number and number to character
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}

'''lambda is essential when you wanna creatye a function for a small period of time and for a very simple task.
Complex lambda function reduces readibility. its works as a normal def in python'''
#Forming an encoding and decoding logic for the maps
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [ ]:
'''torch.long - tensor should be of integer type with 64 bits of precision'''
#Converting array data to tensor as pytorch deals with data in tensor
data = torch.tensor(encode(text), dtype = torch.long)
# print(data[:100])

In [ ]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

In [ ]:
'''Pytorch practise'''
# torch.stack , torch.multinomial, torch.trill, torch.triu, input.T/input.transpose, nn.Linear, torch.cat, F.softmax

In [ ]:
n = int(0.8*len(data))
training_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = training_data if split =='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,) )
    # print(ix)

    x = torch.stack([data[i:i+block_size]for i in ix])
    y = torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x,y

x, y = get_batch('train')

In [ ]:
@torch.no_grad() #ensures that there is no gradient descent happeining ins this code block. There is no training here because no gradients
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [ ]:
'''nn.Embedding is nothing but a lookup table and logits'''

class BiGramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
# so the reason we are doing to make data into 2D, and B is batch_dimensions, T is the size of the time sequence dimesnion, C is the channels
# the reason we need it in 2D is becuase cross_entrpy function accepts only 2 variables.
        return logits, loss

    def generator(self, index, max_new_tokens):
    # Generate new tokens
        for _ in range(max_new_tokens):
            # Forward pass to get logits and loss (we don't use loss here, just for forward computation)
            logits, loss = self.forward(index)
            # Extract the logits for the last token in the sequence
            logits = logits[:, -1, :]
            # Calculate softmax probabilities over the logits to get the probability distribution for the next token
            next_token_probs = F.softmax(logits, dim=-1)
            # Sample from the probability distribution to get the index of the next token
            next_token_index = torch.multinomial(next_token_probs, num_samples=1)
            # Concatenate the index of the next token to the sequence
            index = torch.cat((index, next_token_index), dim=1)
        return index

m = BiGramLanguageModel(vocab_size)

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate, weight_decay=0.01)

In [ ]:
for iter in range(max_iters):
    # print(iter)
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generator(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)